In [30]:
import rioxarray
import numpy as np
import xarray as xr
import pandas as pd

In [22]:
dset = rioxarray.open_rasterio('/Users/marthamorrissey/Downloads/simple_dist_stack_Southern_Rockies.tif',
                               band_as_variable=True)

In [15]:
dset

<xarray.Dataset>
Dimensions:      (x: 16339, y: 28057)
Coordinates:
  * x            (x) float64 -1.183e+06 -1.183e+06 ... -6.932e+05 -6.932e+05
  * y            (y) float64 2.243e+06 2.243e+06 ... 1.402e+06 1.402e+06
    spatial_ref  int64 0
Data variables: (12/22)
    band_1       (y, x) uint8 255 255 255 255 255 255 ... 255 255 255 255 255
    band_2       (y, x) uint8 ...
    band_3       (y, x) uint8 ...
    band_4       (y, x) uint8 ...
    band_5       (y, x) uint8 ...
    band_6       (y, x) uint8 ...
    ...           ...
    band_17      (y, x) uint8 255 255 255 255 255 255 ... 255 255 255 255 255
    band_18      (y, x) uint8 ...
    band_19      (y, x) uint8 ...
    band_20      (y, x) uint8 ...
    band_21      (y, x) uint8 ...
    band_22      (y, x) uint8 ...
Attributes:
    AREA_OR_POINT:  Area

In [14]:
np.unique(dset['band_17'])

array([  0,   1,   2, 255], dtype=uint8)

In [26]:
years = range(1999, 2021)
time_coords = pd.to_datetime([f'{year}-01-01' for year in years])  # Convert years to datetime objects

In [25]:
len(time_coords)

21

In [27]:
for d, i in enumerate(list(dset.data_vars)):
    print(d, i)

0 band_1
1 band_2
2 band_3
3 band_4
4 band_5
5 band_6
6 band_7
7 band_8
8 band_9
9 band_10
10 band_11
11 band_12
12 band_13
13 band_14
14 band_15
15 band_16
16 band_17
17 band_18
18 band_19
19 band_20
20 band_21
21 band_22


In [28]:
time_lst = []

for i, v in enumerate(dset.data_vars):
    dset_tmp = dset[v].to_dataset().assign_coords(time=time_coords[i])
    time_lst.append(dset_tmp)

In [29]:
time_lst

[<xarray.Dataset>
 Dimensions:      (x: 16339, y: 28057)
 Coordinates:
   * x            (x) float64 -1.183e+06 -1.183e+06 ... -6.932e+05 -6.932e+05
   * y            (y) float64 2.243e+06 2.243e+06 ... 1.402e+06 1.402e+06
     spatial_ref  int64 0
     time         datetime64[ns] 1999-01-01
 Data variables:
     band_1       (y, x) uint8 ...,
 <xarray.Dataset>
 Dimensions:      (x: 16339, y: 28057)
 Coordinates:
   * x            (x) float64 -1.183e+06 -1.183e+06 ... -6.932e+05 -6.932e+05
   * y            (y) float64 2.243e+06 2.243e+06 ... 1.402e+06 1.402e+06
     spatial_ref  int64 0
     time         datetime64[ns] 2000-01-01
 Data variables:
     band_2       (y, x) uint8 ...,
 <xarray.Dataset>
 Dimensions:      (x: 16339, y: 28057)
 Coordinates:
   * x            (x) float64 -1.183e+06 -1.183e+06 ... -6.932e+05 -6.932e+05
   * y            (y) float64 2.243e+06 2.243e+06 ... 1.402e+06 1.402e+06
     spatial_ref  int64 0
     time         datetime64[ns] 2001-01-01
 Data variables

In [1]:
time_lst[0]

NameError: name 'time_lst' is not defined

In [31]:
dset_format = xr.concat(time_lst, dim='time')

: 